In [1]:
!git clone https://github.com/Tanveer2719/NIDS_Coursework.git

Cloning into 'NIDS_Coursework'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 123 (delta 42), reused 118 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (123/123), 141.67 KiB | 2.72 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [2]:
%cd /kaggle/working/NIDS_Coursework/My_Code
!ls

/kaggle/working/NIDS_Coursework/My_Code
classification.py    main.ipynb     record_level_embedding.py
featureSelection.py  preprocess.py  transformer.py


In [3]:
%rm -r __pycache__

rm: cannot remove '__pycache__': No such file or directory


In [4]:
!git pull origin main

From https://github.com/Tanveer2719/NIDS_Coursework
 * branch            main       -> FETCH_HEAD
Already up to date.


In [5]:
import pandas as pd
import numpy as np

path = "/kaggle/input/unsw-nb-15/UNSW-NB15_combined.csv"

df = pd.read_csv(path)

print(df.shape)
df = df.drop(columns=['id', 'label'])
print(df.shape)

print(f"🧾 Unique class labels: {df['attack_cat'].unique()}")
df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Normal' else 1)

print(f'Unique values for label {df["label"].unique()}')

df = df.drop(columns=["attack_cat"])
print(f'new shape{df.shape}')

print(df.dtypes)


(257673, 45)
(257673, 43)
🧾 Unique class labels: ['Normal' 'Backdoor' 'Analysis' 'Fuzzers' 'Shellcode' 'Reconnaissance'
 'Exploits' 'DoS' 'Worms' 'Generic']
Unique values for label [0 1]
new shape(257673, 43)
dur                  float64
proto                 object
service               object
state                 object
spkts                  int64
dpkts                  int64
sbytes                 int64
dbytes                 int64
rate                 float64
sttl                   int64
dttl                   int64
sload                float64
dload                float64
sloss                  int64
dloss                  int64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                   int64
stcpb                  int64
dtcpb                  int64
dwin                   int64
tcprtt               float64
synack               float64
ackdat               float64
smean                  int64
dmean   

In [6]:
# perform preprocessing

from preprocess import Preprocess

# separate the features and labels so that the labesl are not encoded
labels = df['label']
features = df.drop(columns=['label'])

pp = Preprocess()
processed_features = pp.fit_transform_df_auto(df = features,n_categorical_levels=32, expected_categorical_format='onehot')

print(processed_features.shape)
processed_features.head()

Encoding the 32 levels for proto
Encoding the 13 levels for service
Encoding the 11 levels for state
(257673, 95)


,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_2,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11
0,0.027889,0.193225,0.172921,0.331292,0.312312,0.312595,0.997874,1.000000,0.424562,0.534539,...,False,False,False,False,False,False,False,False,False,False
1,0.121803,0.284598,0.393619,0.398471,0.645181,0.316704,0.747160,0.998579,0.401347,0.775718,...,False,False,False,False,False,False,False,False,False,False
2,0.234590,0.224248,0.304405,0.353884,0.574953,0.196832,0.747160,0.998579,0.326962,0.650937,...,False,False,False,False,False,False,False,False,False,False
3,0.239956,0.267974,0.275582,0.388675,0.402879,0.194440,0.747160,0.998579,0.351625,0.479722,...,False,False,False,False,False,False,False,False,False,False
4,0.090294,0.248312,0.209072,0.378428,0.339064,0.256038,0.999294,0.998579,0.402218,0.489854,...,False,False,False,False,False,False,False,False,False,False


In [7]:
# concat the features and the labels
processed_full_df = pd.concat([processed_features, labels], axis=1)
processed_full_df.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11,label
0,0.027889,0.193225,0.172921,0.331292,0.312312,0.312595,0.997874,1.000000,0.424562,0.534539,...,False,False,False,False,False,False,False,False,False,0
1,0.121803,0.284598,0.393619,0.398471,0.645181,0.316704,0.747160,0.998579,0.401347,0.775718,...,False,False,False,False,False,False,False,False,False,0
2,0.234590,0.224248,0.304405,0.353884,0.574953,0.196832,0.747160,0.998579,0.326962,0.650937,...,False,False,False,False,False,False,False,False,False,0
3,0.239956,0.267974,0.275582,0.388675,0.402879,0.194440,0.747160,0.998579,0.351625,0.479722,...,False,False,False,False,False,False,False,False,False,0
4,0.090294,0.248312,0.209072,0.378428,0.339064,0.256038,0.999294,0.998579,0.402218,0.489854,...,False,False,False,False,False,False,False,False,False,0


In [28]:
from featureSelection import KBestFeatureSelector

kbest = KBestFeatureSelector(k=60)  # Set k to your desired number of features
selected_mask, selected_feature = kbest.select_features(df=processed_full_df, target_column='label', verbose=True)

selected_feature


🔍 Starting SelectKBest feature selection (k = 60) using mutual_info_classif...
✅ Selected top 60 features out of 95
📌 Selected features:
   1. dur
   2. spkts
   3. dpkts
   4. sbytes
   5. dbytes
   6. rate
   7. sttl
   8. dttl
   9. sload
  10. dload
  11. sloss
  12. dloss
  13. sinpkt
  14. dinpkt
  15. sjit
  16. djit
  17. swin
  18. stcpb
  19. dtcpb
  20. dwin
  21. tcprtt
  22. synack
  23. ackdat
  24. smean
  25. dmean
  26. response_body_len
  27. ct_srv_src
  28. ct_state_ttl
  29. ct_dst_ltm
  30. ct_src_dport_ltm
  31. ct_dst_sport_ltm
  32. ct_dst_src_ltm
  33. ct_flw_http_mthd
  34. ct_src_ltm
  35. ct_srv_dst
  36. is_sm_ips_ports
  37. proto_1
  38. proto_2
  39. proto_3
  40. proto_4
  41. proto_5
  42. proto_6
  43. proto_7
  44. proto_8
  45. proto_11
  46. proto_18
  47. proto_27
  48. proto_29
  49. service_1
  50. service_2
  51. service_3
  52. service_5
  53. service_7
  54. service_8
  55. service_10
  56. state_1
  57. state_2
  58. state_3
  59. state_4


Index(['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
       'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit',
       'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean',
       'dmean', 'response_body_len', 'ct_srv_src', 'ct_state_ttl',
       'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports',
       'proto_1', 'proto_2', 'proto_3', 'proto_4', 'proto_5', 'proto_6',
       'proto_7', 'proto_8', 'proto_11', 'proto_18', 'proto_27', 'proto_29',
       'service_1', 'service_2', 'service_3', 'service_5', 'service_7',
       'service_8', 'service_10', 'state_1', 'state_2', 'state_3', 'state_4',
       'state_11'],
      dtype='object')

In [29]:
columns_to_keep = selected_feature

In [30]:
df_selected = processed_full_df[columns_to_keep].copy()
df_selected.shape

(257673, 60)

In [31]:
df_selected.dtypes

dur                  float64
spkts                float64
dpkts                float64
sbytes               float64
dbytes               float64
rate                 float64
sttl                 float64
dttl                 float64
sload                float64
dload                float64
sloss                float64
dloss                float64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                 float64
stcpb                float64
dtcpb                float64
dwin                 float64
tcprtt               float64
synack               float64
ackdat               float64
smean                float64
dmean                float64
response_body_len    float64
ct_srv_src           float64
ct_state_ttl         float64
ct_dst_ltm           float64
ct_src_dport_ltm     float64
ct_dst_sport_ltm     float64
ct_dst_src_ltm       float64
ct_flw_http_mthd     float64
ct_src_ltm           float64
ct_srv_dst    

In [32]:
numerical_columns = [col for col in columns_to_keep 
                     if not (col.startswith("proto_") or 
                             col.startswith("service_") or 
                             col.startswith("state_"))]


In [33]:
from record_level_embedding import RecordLevelEmbedder

embedder = RecordLevelEmbedder(selected_df = df_selected, numerical_columns = numerical_columns, embed_dimension= 64)

embedded_df = embedder.transform_to_df()

embedded_df.head()

[INFO] Initializing RecordLevelEmbedder...
[INFO] Detecting categorical blocks...
[INFO] Detected 3 categorical blocks:
   - proto: 12 columns
   - service: 7 columns
   - state: 5 columns
[INFO] Embedder initialized with embedding dimension 64.
[INFO] Building Keras embedding model...
   - Adding numerical input: dur
   - Adding numerical input: spkts
   - Adding numerical input: dpkts
   - Adding numerical input: sbytes
   - Adding numerical input: dbytes
   - Adding numerical input: rate
   - Adding numerical input: sttl
   - Adding numerical input: dttl
   - Adding numerical input: sload
   - Adding numerical input: dload
   - Adding numerical input: sloss
   - Adding numerical input: dloss
   - Adding numerical input: sinpkt
   - Adding numerical input: dinpkt
   - Adding numerical input: sjit
   - Adding numerical input: djit
   - Adding numerical input: swin
   - Adding numerical input: stcpb
   - Adding numerical input: dtcpb
   - Adding numerical input: dwin
   - Adding numeri

,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_54,embed_55,embed_56,embed_57,embed_58,embed_59,embed_60,embed_61,embed_62,embed_63
0,0.220751,-0.247311,-0.303010,0.197911,-0.709915,-0.154194,0.157538,-0.383584,-0.182741,0.083476,...,0.424398,0.128992,0.397317,-0.194838,0.076148,0.314496,-0.099968,-0.023741,0.554119,-0.469880
1,0.404968,-0.204791,-0.220928,-0.012596,-1.060443,-0.106696,0.072947,-0.537508,-0.308040,0.298800,...,0.051974,0.165191,0.702790,-0.154988,0.051972,0.544853,-0.115924,-0.186948,0.581429,-0.506521
2,0.326006,-0.100894,-0.122201,0.163734,-0.912672,-0.166526,0.079399,-0.546414,-0.440572,0.345125,...,0.121017,0.276450,0.488343,-0.384932,0.261505,0.457885,-0.167988,-0.055049,0.495147,-0.461951
3,-0.007831,-0.103884,-0.052118,0.102106,-0.594185,-0.042180,0.033312,-0.316425,-0.341915,0.288660,...,0.450286,0.306075,0.188856,-0.186160,0.003236,0.185356,-0.015589,-0.189982,0.368416,-0.291898
4,0.341280,-0.127741,0.051448,-0.024095,-0.940199,-0.281057,0.060937,-0.331868,-0.549533,0.113156,...,0.032622,0.374138,0.415411,-0.400421,0.182955,0.739492,-0.206423,0.142796,0.681797,-0.510987


In [34]:
# Binary classification
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model


print(processed_full_df['label'].unique())

binary_target = (processed_full_df['label'] != 0).astype(int).values
unique, counts = np.unique(binary_target, return_counts=True)
print("Classes and their counts:")
for cls, cnt in zip(unique, counts):
    print(f"Class {cls}: {cnt}")

[0 1]
Classes and their counts:
Class 0: 93000
Class 1: 164673


In [38]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from transformer import BasicTransformer
from classification import LastTokenClassificationHead
from sklearn.model_selection import train_test_split
import numpy as np  # Just in case

# Input data
X_embed = embedded_df.values                   # shape: (num_samples, embed_dim)
X_embed_seq = tf.expand_dims(X_embed, axis=1)  # shape: (batch_size, 1, embed_dim)

input_layer = Input(shape=(1, X_embed.shape[1]), name="record_input")

# Transformer block
transformer = BasicTransformer(
    n_layers=2,
    internal_size=128,
    n_heads=4,
    verbose=True  # See progress!
)

x = transformer.apply(input_layer, training=True)

# Classification head
classification_head = LastTokenClassificationHead()
x = classification_head.apply(x)

# MLP head
for i, layer_size in enumerate([128]):
    x = Dense(layer_size, activation="relu", name=f"classification_mlp_{i}_{layer_size}")(x)
    x = Dropout(0.1)(x)

# Output layer for binary classification
output_layer = Dense(1, activation="sigmoid", name="binary_output")(x)

# Build the model
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

# Compile the model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
    jit_compile=True
)

# Prepare data
X_np = X_embed_seq.numpy() if hasattr(X_embed_seq, 'numpy') else np.array(X_embed_seq)

X_train, X_val, y_train, y_val = train_test_split(
    X_np, binary_target, test_size=0.2, random_state=42, stratify=binary_target
)

# Define early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Train the model
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop]
)


[BasicTransformer] Applying Encoder Block 1/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Applying Encoder Block 2/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Final output shape: (None, 1, 64)


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ record_input (InputLayer)       │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_0_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_1_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ slice_last (Lambda)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification_mlp_0_128        │ (None, 128)            │         8,320 │
│ (Dense)                         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ binary_output (Dense)           │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 307,457 (1.17 MB)

 Trainable params: 307,457 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9020 - loss: 0.1958[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 43s 5ms/step - accuracy: 0.9020 - loss: 0.1958 - val_accuracy: 0.9221 - val_loss: 0.1619
Epoch 2/50
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.9169 - loss: 0.1646 - val_accuracy: 0.9233 - val_loss: 0.1528
Epoch 3/50
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.9207 - loss: 0.1585 - val_accuracy: 0.9196 - val_loss: 0.1509
Epoch 4/50
6442/6442 

In [39]:
# evaluate the model
results = model.evaluate(X_val, y_val)
print(f"Validation loss: {results[0]:.4f}, Validation accuracy: {results[1]:.4f}")


1611/1611 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9375 - loss: 0.1256
Validation loss: 0.1262, Validation accuracy: 0.9372


In [40]:
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

# Predict probabilities
y_pred_prob = model.predict(X_val)
y_pred = (y_pred_prob >= 0.5).astype(int).flatten()

# F1
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1:.4f}")

# Confusion matrix
cm = confusion_matrix(y_val, y_pred, labels=[0, 1])
print("Confusion Matrix:\n", cm)

if cm.shape == (2, 2):
    tn, fp, fn, tp = cm.ravel()
    detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    print(f"Detection Rate (Recall): {detection_rate:.4f}")
    print(f"False Alarm Rate: {false_alarm_rate:.4f}")
else:
    print("Confusion matrix does not contain both classes!")
    print("Check your labels: y_val:", np.unique(y_val), " y_pred:", np.unique(y_pred))


[EncoderBlock] Input shape: (32, 1, 64)
[EncoderBlock] Output shape: (32, 1, 64)
[EncoderBlock] Input shape: (32, 1, 64)
[EncoderBlock] Output shape: (32, 1, 64)
1593/1611 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
1611/1611 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
F1 Score: 0.9506
Confusion Matrix:
 [[17140  1460]
 [ 1777 31158]]
Detection Rate (Recall): 0.9460
False Alarm Rate: 0.0785
